# Pothole Detection 1

<!DOCTYPE html>
<html>

<body>
<img style="display: block;
            padding-right: 5%;
            width: 50%;" src="https://media4.giphy.com/media/v1.Y2lkPTc5MGI3NjExaWJ2YzFjMjF0bTU1MTc4a3FyN2Z6aGJoYWg4ZHoybTk2aXI3bmxvbiZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9cw/FT6adJ0iu560vH3Kqy/giphy.gif"/>

<hr>
<hr>
<h1 style="font-family:Comic Sans MS;" align="center">Computer Vision Project</h1>
<h1 style="font-family:Comic Sans MS;" align="center">Department of Computer Engineering</h1>
<h1 style="font-family:Comic Sans MS;" align="center">Hacettepe University</h1>
<h1 style="font-family:Comic Sans MS;" align="center">Ankara, Turkey</h1>
<hr>
<hr>
</body>
    
</html>



In [1]:
import numpy as np
import cv2
import os
import random
from ultralytics import YOLO

In [2]:
video_folder = 'train/rgb'

video_files = [f for f in os.listdir(video_folder) if os.path.isfile(os.path.join(video_folder, f))]
num_videos = len(video_files)

print(num_videos)


373


In [22]:
selected_videos = random.sample(video_files, 10)

print("Randomly select 10 video:")
for video in selected_videos:
    print(video)

Randomly select 10 video:
0038.mp4
0328.mp4
0233.mp4
0006.mp4
0317.mp4
0516.mp4
0223.mp4
0055.mp4
0035.mp4
0010.mp4


In [5]:
def compute_flow(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (3, 3), 5)
    gray2 = cv2.GaussianBlur(gray2, (3, 3), 5)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, pyr_scale=0.75, levels=3, winsize=5, iterations=3, poly_n=10, poly_sigma=1.2, flags=0)
    return flow


In [6]:
def get_motion_mask(flow_mag, motion_thresh=1, kernel=np.ones((7,7))):
    motion_mask = np.uint8(flow_mag > motion_thresh) * 255
    motion_mask = cv2.erode(motion_mask, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_OPEN, kernel, iterations=1)
    motion_mask = cv2.morphologyEx(motion_mask, cv2.MORPH_CLOSE, kernel, iterations=3)
    return motion_mask


In [7]:
def compute_flow_histo(frame1, frame2):
    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    # Apply Histogram Equalization and increase contrast
    gray1 = cv2.equalizeHist(gray1)
    gray2 = cv2.equalizeHist(gray2)

    gray1 = cv2.GaussianBlur(gray1, (3, 3), 5)
    gray2 = cv2.GaussianBlur(gray2, (3, 3), 5)
    flow = cv2.calcOpticalFlowFarneback(gray1, gray2, None, pyr_scale=0.75, levels=3, winsize=5, iterations=3, poly_n=10, poly_sigma=1.2, flags=0)
    return flow


In [8]:
def apply_color_space_conversion(frame):
    # Convert image to HSV color space
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
    
    # Getting better contrast by taking the V (value) channel in color space
    v_channel = hsv[:,:,2]
    
    # Apply histogram equalization
    equalized_v_channel = cv2.equalizeHist(v_channel)
    
    # Reassemble
    hsv[:,:,2] = equalized_v_channel
    
    # Converting from HSV to BGR
    result_frame = cv2.cvtColor(hsv, cv2.COLOR_HSV2BGR)
    
    return result_frame
# KAYNAK: https://staff.fnwi.uva.nl/r.vandenboomgaard/ComputerVision/LectureNotes/IP/PointOperators/HistogramEqualization.html

# Dense Optical Flow & Histogram Equalization # 

In [10]:
def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    
    if not ret:
        print(f"Video {video_path} not found.")
        return
    
    while cap.isOpened():
        ret, frame2 = cap.read()
        ret, frame3 = cap.read()
        if not ret:
            break

        # Calculating Dense Optical Flow
        flow = compute_flow(frame1, frame2)
        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Calculating Dense Optical Flow with Histogram Equalization
        flow_histo = compute_flow_histo(frame1, frame3)
        mag_histo, _ = cv2.cartToPolar(flow_histo[..., 0], flow_histo[..., 1])
        
        # Create motion mask
        min_thresh = 0.3
        max_thresh = 1
        motion_thresh = np.c_[np.linspace(min_thresh, max_thresh, frame1.shape[0])].repeat(frame1.shape[1], axis=-1)
        mask = get_motion_mask(mag, motion_thresh=motion_thresh)
        
        mask_histo = get_motion_mask(mag_histo, motion_thresh=motion_thresh)

        frame_resized = cv2.resize(frame2, (640, 480))
        mask_resized = cv2.resize(mask, (640, 480))

        frame3_resized = cv2.resize(frame3, (640, 480))
        mask_histo_resized = cv2.resize(mask_histo, (640, 480))
        
        # Results of Flow w/ Histogram Equalization
        cv2.imshow("Frame", frame_resized)
        cv2.imshow("Motion Mask", mask_resized)
        cv2.imshow("Motion Mask Histo", mask_histo_resized)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame1 = frame2

    cap.release()
    cv2.destroyAllWindows()

for video in selected_videos:
    video_path = os.path.join(video_folder, video)
    print(f"{video} processing...")
    process_video(video_path)


0215.mp4 processing...


# Color space transformation process # 

In [28]:

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    
    if not ret:
        print(f"Video {video_path} not found.")
        return
    
    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        # Color space transformation process
        processed_frame1 = apply_color_space_conversion(frame1)
        processed_frame2 = apply_color_space_conversion(frame2)

        # Detection
        # detected_frame2 = detect_objects(processed_frame2)

        # Calculating Dense Optical Flow
        flow = compute_flow_histo(processed_frame1, processed_frame2)
        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        # Create motion mask
        min_thresh = 0.3
        max_thresh = 1
        motion_thresh = np.c_[np.linspace(min_thresh, max_thresh, frame1.shape[0])].repeat(frame1.shape[1], axis=-1)
        mask = get_motion_mask(mag, motion_thresh=motion_thresh)

        # Resize frames and mask
        frame1_resized = cv2.resize(frame1, (640, 480))
        frame2_resized = cv2.resize(frame2, (640, 480))
        processed_frame2_resized = cv2.resize(processed_frame2, (640, 480))
        mask_resized = cv2.resize(mask, (640, 480))

        # Results
        cv2.imshow("Original Frame", frame1_resized)
        cv2.imshow("Color space transformed Frame", processed_frame2_resized)
        cv2.imshow("Motion Mask", mask_resized)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame1 = frame2

    cap.release

    cap.release()
    cv2.destroyAllWindows()

for video in selected_videos:
    video_path = os.path.join(video_folder, video)
    print(f"{video} processing...")
    process_video(video_path)


0328.mp4 processing...
0215.mp4 processing...
0554.mp4 processing...
0516.mp4 processing...
0134.mp4 processing...
0328.mp4 processing...
0325.mp4 processing...


# Non-Maximum Supression & Detection # 

In [27]:

model = YOLO("seg_v1.pt")


def detect_objects(frame):
    # Call YOLO model for output and get the output
    results = model.predict(frame)
    
    # process boxes in predictions
    for result in results:
        for bbox in result.boxes.xyxy:
            # Check the boxes
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"Class: Pothole"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    return frame

def get_motion_mask_x(magnitude, motion_thresh):
    mask = np.zeros_like(magnitude, dtype=np.uint8)
    mask[magnitude > motion_thresh] = 255
    return mask

def non_maximum_suppression(mask, kernel_size=5):
    dilated = cv2.dilate(mask, np.ones((kernel_size, kernel_size), np.uint8))
    nms_mask = (mask == dilated) & (mask > 0)
    return nms_mask.astype(np.uint8) * 255

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    ret, frame1 = cap.read()
    
    if not ret:
        print(f"Video {video_path} not found.")
        return
    
    frame1 = apply_color_space_conversion(frame1)
    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        processed_frame2 = apply_color_space_conversion(frame2)
        detected_frame = detect_objects(frame1)
        flow = compute_flow_histo(frame1, processed_frame2)
        mag, _ = cv2.cartToPolar(flow[..., 0], flow[..., 1])

        min_thresh = 0.3
        max_thresh = 1
        motion_thresh = np.c_[np.linspace(min_thresh, max_thresh, frame1.shape[0])].repeat(frame1.shape[1], axis=-1)
        mask = get_motion_mask(mag, motion_thresh=motion_thresh)
        mask_x = get_motion_mask_x(mag, motion_thresh=motion_thresh)

        nms_mask = non_maximum_suppression(mask_x)

        processed_frame2_resized = cv2.resize(processed_frame2, (640, 480))
        mask_resized = cv2.resize(mask, (640, 480))
        nms_mask_resized = cv2.resize(nms_mask, (640, 480))
        detected_frame_resized = cv2.resize(detected_frame, (640, 480))

        cv2.imshow("Processed Frame", processed_frame2_resized)
        cv2.imshow("Motion Mask", mask_resized)
        cv2.imshow("NMS Motion Mask", nms_mask_resized)
        cv2.imshow("DETECTIOM",detected_frame_resized)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        frame1 = frame2

    cap.release()
    cv2.destroyAllWindows()


for video in selected_videos:
    video_path = os.path.join(video_folder, video)
    print(f"{video} processing...")
    process_video(video_path)


0328.mp4 processing...

0: 512x512 6 Potholes, 324.5ms
Speed: 6.0ms preprocess, 324.5ms inference, 27.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 343.2ms
Speed: 4.0ms preprocess, 343.2ms inference, 10.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 347.3ms
Speed: 5.0ms preprocess, 347.3ms inference, 10.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 379.0ms
Speed: 5.0ms preprocess, 379.0ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 337.6ms
Speed: 6.1ms preprocess, 337.6ms inference, 8.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 335.7ms
Speed: 6.0ms preprocess, 335.7ms inference, 9.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 342.8ms
Speed: 4.0ms preprocess, 342.8ms inference, 11.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 2 Potholes, 356.2ms
Speed: 6.0ms preprocess, 356.2m

# Detection # 

In [24]:
import cv2

video_path = "train/rgb/0215.mp4"
cap = cv2.VideoCapture(video_path)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Call YOLO model for output and get the output
    results = model.predict(frame)
    
    # Manipulate boxes in predictions
    for result in results:
        for bbox in result.boxes.xyxy:
            # Kutuyu işaretle
            x1, y1, x2, y2 = map(int, bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            text = f"Class: Pothole"
            cv2.putText(frame, text, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            
    cv2.imshow("YOLOv8e Object Detection", frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()



0: 512x512 1 Pothole, 234.5ms
Speed: 4.1ms preprocess, 234.5ms inference, 5.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 271.8ms
Speed: 4.0ms preprocess, 271.8ms inference, 6.1ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 270.6ms
Speed: 4.0ms preprocess, 270.6ms inference, 5.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 331.3ms
Speed: 5.0ms preprocess, 331.3ms inference, 6.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 252.6ms
Speed: 4.0ms preprocess, 252.6ms inference, 4.1ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 246.4ms
Speed: 4.0ms preprocess, 246.4ms inference, 4.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 248.3ms
Speed: 6.4ms preprocess, 248.3ms inference, 5.0ms postprocess per image at shape (1, 3, 512, 512)

0: 512x512 1 Pothole, 237.5ms
Speed: 4.0ms preprocess, 237.5ms inference, 2.9ms postprocess per 

<img align="center" height="447" src="https://media2.giphy.com/media/v1.Y2lkPTc5MGI3NjExc2FxbDZnaHhmaG83eGZ5YzZycm85eG56ajhlaHR2NmRhZzhqemxpaSZlcD12MV9pbnRlcm5hbF9naWZfYnlfaWQmY3Q9Zw/fjgqYjVkzfQ9a/giphy.gif"/>

```javascript
Hacettepe University @2024;
```